In [1]:
import gc, os, cv2, PIL, torch
import torchvision as tv
import torch.nn as nn
import torchsummary as ts
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
import random

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index]
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

In [3]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()

# Retrieving the images and their labels
for i in range(classes):
    path = os.path.join(cur_path, 'Train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(os.path.join(path, a))
            image = image.resize((32, 32))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

In [4]:
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

# Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=100)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(39209, 32, 32, 3) (39209,)
(31367, 32, 32, 3) (7842, 32, 32, 3) (31367,) (7842,)


In [5]:
class RandomGrayscale(object):
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, img):
        if random.random() < self.p:
            img = transforms.functional.rgb_to_grayscale(img, num_output_channels=3)
        return img

In [6]:
# Applying transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    RandomGrayscale(p=0.2),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [7]:
model = nn.Sequential(
                        # 1st convolutional network Layers
                        nn.Conv2d(3,16,(2,2),(1,1),'same'),   # Convolution
                        nn.BatchNorm2d(16),                   # Normalization 
                        nn.ReLU(True),                       # Activation
                        nn.MaxPool2d((2,2)),                 # Pooling
    
                        # 2nd convolutional network Layers
                        nn.Conv2d(16,32,(2,2),(1,1),'same'),  # Convolution
                        nn.BatchNorm2d(32),                  # Normalization 
                        nn.ReLU(True),                       # Activation
                        nn.MaxPool2d((2,2)),                 # Pooling
    
                        # 3rd convolutional network Layers
                        nn.Conv2d(32,64,(2,2),(1,1),'same'), # Convolution
                        nn.BatchNorm2d(64),                  # Normalization 
                        nn.ReLU(True),                       # Activation
                        nn.MaxPool2d((2,2)),                 # Pooling

                        # Flatten Data
                        nn.Flatten(),                        # Flatten
    
                        # feed forward Layers
                        nn.Linear(1024,256),                  # Linear 
                        nn.ReLU(True),                       # Activation
                        nn.Linear(256,43)                    # Linear 
                    )

# Send model to Cuda Memory
model = model.to(torch.device('cuda'),non_blocking=True)
# For Model Summary
ts.summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             208
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
         MaxPool2d-4           [-1, 16, 16, 16]               0
            Conv2d-5           [-1, 32, 16, 16]           2,080
       BatchNorm2d-6           [-1, 32, 16, 16]              64
              ReLU-7           [-1, 32, 16, 16]               0
         MaxPool2d-8             [-1, 32, 8, 8]               0
            Conv2d-9             [-1, 64, 8, 8]           8,256
      BatchNorm2d-10             [-1, 64, 8, 8]             128
             ReLU-11             [-1, 64, 8, 8]               0
        MaxPool2d-12             [-1, 64, 4, 4]               0
          Flatten-13                 [-1, 1024]               0
           Linear-14                  [

E:\anaconda3\envs\pt\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


In [8]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Number of epochs
num_epochs = 20

# Training the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Оберните train_loader с tqdm для визуализации прогресса
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training")
    for i, (images, labels) in enumerate(pbar):
        # Transfer to GPU
        images, labels = images.to(torch.device('cuda')), labels.to(torch.device('cuda'))

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({'loss': running_loss / (i + 1)})
    
    # Validation of the model
    model.eval()
    correct = 0
    total = 0
    val_pbar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation")
    with torch.no_grad():
        for images, labels in val_pbar:
            images, labels = images.to(torch.device('cuda')), labels.to(torch.device('cuda'))
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_pbar.set_postfix({'accuracy': 100 * correct / total})

    print(f'Accuracy of the model on the {len(test_dataset)} test images: {100 * correct / total:.2f}%')

Epoch 1/20 Validation: 100%|██████████| 31/31 [00:01<00:00, 16.94it/s, accuracy=64.6]


Accuracy of the model on the 7842 test images: 64.60%


Epoch 2/20 Validation: 100%|██████████| 31/31 [00:01<00:00, 18.38it/s, accuracy=87.6]


Accuracy of the model on the 7842 test images: 87.63%


Epoch 3/20 Validation: 100%|██████████| 31/31 [00:01<00:00, 17.27it/s, accuracy=93.3]


...

Accuracy of the model on the 7842 test images: 98.55%


Epoch 19/20 Validation: 100%|██████████| 31/31 [00:01<00:00, 17.91it/s, accuracy=98.7]


Accuracy of the model on the 7842 test images: 98.72%


Epoch 20/20 Validation: 100%|██████████| 31/31 [00:01<00:00, 18.09it/s, accuracy=98.9]

Accuracy of the model on the 7842 test images: 98.85%


In [9]:
torch.save(model.state_dict(), f"best_model_epoch_{777}.pth")